# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [94]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [95]:
weather_df = pd.read_csv("../output_data/cities.csv")
weather_df.head()

,City,Lat,Lng,MaxTemp,Humidity,Cloudiness,Wind Speed,Country,Date
0,b'Upernavik',72.79,-56.15,35.46,95,100,13.73,GL,1602983180
1,b'Busselton',-33.65,115.33,64.00,66,0,1.99,AU,1602983102
2,"b""Oktyabr'skoye""",43.06,44.74,42.80,93,0,6.71,RU,1602983180
3,b'Mataura',-46.19,168.86,53.01,64,98,11.01,NZ,1602983180
4,b'Atambua',-9.11,124.89,87.28,46,74,2.75,ID,1602983180


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [96]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)

In [97]:
# Store latitude and longitude in locations
locations = weather_df[["Lat", "Lng"]]

# convert to float
rating = weather_df["Humidity"].astype(float)

In [98]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=rating, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=2)
# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [99]:
# Create ideal weather conditions df
# Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

ideal_temp_min = 80.0
ideal_wind_max = 11.0
ideal_cloud = 0

ideal_weather_df = weather_df[weather_df['MaxTemp'] > ideal_temp_min]
ideal_weather_df = ideal_weather_df[ideal_weather_df['Wind Speed'] < ideal_wind_max]
ideal_weather_df = ideal_weather_df[ideal_weather_df['Cloudiness'] == ideal_cloud]
ideal_weather_df


,City,Lat,Lng,MaxTemp,Humidity,Cloudiness,Wind Speed,Country,Date
92,b'Araouane',18.90,-3.53,89.33,16,0,8.63,ML,1602983193
149,b'Diffa',13.32,12.61,85.24,19,0,5.30,NE,1602983201
185,b'Bilma',18.69,12.92,82.31,17,0,1.77,NE,1602983207
205,b'Gashua',12.87,11.05,85.68,21,0,4.65,NG,1602983208
304,b'Sahuaripa',29.05,-109.23,84.16,38,0,5.95,MX,1602983222
322,b'Tessalit',20.20,1.01,82.20,17,0,8.14,ML,1602983144
339,b'Adrar',20.50,-10.07,86.05,23,0,10.31,MR,1602982931
466,b'Taoudenni',22.68,-3.98,82.63,21,0,7.56,ML,1602983246
551,b'Umluj',25.02,37.27,82.89,41,0,6.78,SA,1602983259


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [100]:
hotels = list()
hotel_city = list()
hotel_country = list()
hotel_lat = list()
hotel_lng = list()

for i in ideal_weather_df.index:

    # geocoordinates
    target_coordinates = str(ideal_weather_df['Lat'][i]) + "," + str(ideal_weather_df['Lng'][i])
    target_radius = 5000
    target_type = "lodging"

    # set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "radius": target_radius,
        "type": target_type,
        "key": g_key
    }

    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    response = requests.get(base_url, params=params)
    
    # convert response to json
    places_data = response.json()

    # Print the json (pretty printed)
    #print(json.dumps(places_data, indent=4, sort_keys=True))
    
    # If there is a hotel nearby, store the hotel name, city and country in hotel_df
    if places_data["status"] == "OK":
        hotels.append(places_data["results"][0]["name"])
        hotel_city.append(ideal_weather_df["City"][i])
        hotel_country.append(ideal_weather_df["Country"][i])
        hotel_lat.append(ideal_weather_df["Lat"][i])
        hotel_lng.append(ideal_weather_df["Lng"][i])

In [101]:
# Create the pandas DataFrame 
hotel_df = pd.DataFrame({"Hotel Name":hotels,
                         "City":hotel_city,
                         "Country":hotel_country,
                         "Lat":hotel_lat,
                         "Lng":hotel_lng                
                        })
hotel_df

,Hotel Name,City,Country,Lat,Lng
0,ProDAF/Diffa,b'Diffa',NE,13.32,12.61
1,Expeditions Ténére voyages,b'Bilma',NE,18.69,12.92
2,Forte Oil - Old Lorry Park,b'Gashua',NG,12.87,11.05
3,Estatua de Sahuaripa,b'Sahuaripa',MX,29.05,-109.23
4,Moon light Furnished Units,b'Umluj',SA,25.02,37.27


In [102]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [103]:
# Add marker layer ontop of heat map
fig = gmaps.figure()
markers=gmaps.marker_layer(locations,info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))